In [4]:
import random
from enum import Enum

## ENUMS FOR CARD CLASS

In [52]:
# Define Card Type Enum
class CardType(Enum):
    COMBAT = 'Combat Card'
    NON_COMBAT = 'Non-Combat Card'
    DRAGON_BALL = 'Dragon Ball'
    PERSONALITY = 'Personality'
    MASTERY = 'Mastery'
    
class CombatCardType(Enum):
    ENERGY_COMBAT = 'Energy Combat'
    PHYSICAL_COMBAT = 'Physical Combat'
    EVENT_COMBAT = 'Event Combat'

class NonCombatCardType(Enum):
    SETUP = 'Non-Combat: Setup'
    DRILL = 'Non-Combat: Drill'

class PersonalityCardType(Enum):
    HERO = 'Hero'
    VILLAIN = 'Villain'
    ROGUE = 'Rogue'
    ALLY = 'Ally'    


# Define Card Effect Enum
class EffectType(Enum):
    ATTACK = 'Attack Type Card' # Used for performing attacks
    DEFENSE = 'Defense Type Card' # Used for defending against attacks
    ACTIVE_POWER = 'Activate Power' # Activates some effect immediately
    CONSTANT_POWER = 'Constant Power' # Stays in play and applies some effect consinuously
    
    
# Define Attack Type Enum
class AttackType(Enum):
    ENERGY_ATTACK = 'Energy Attack'
    PHYSICAL_ATTACK = 'Physical Attack'
    

# Define Card Style Enum
class CardStyle(Enum):
    SAIYAN_STYLE = "Saiyan Style"
    NAMEKIAN_STYLE = "Namekian Style"
    RED_STYLE = "Red Style"
    BLACK_STYLE = "Black Style"
    BLUE_STYLE = "Blue Style"
    ORANGE_STYLE = "Orange Style"
    FREESTYLE = "Freestyle"
    
    
# Define Card Rarity ENUM
class CardRarity(Enum):
    COMMON = 'Common'
    UNCOMMON = 'Uncommon'
    RARE = 'Rare'
    ULTRA_RATE = 'Ultra Rare'
    STARTER = 'Starter'
    
    
# Define Set Name ENUM
class SetName(Enum):
    SET_SAIYAN_SAGA = 'Saiyan Saga'
    SET_NAMEK_SAGA = 'Namek Saga'
    # Add More
    
    
# Define Named Character ENUM
class NamedCharacter(Enum):
    GOKU = 'Goku'
    VEGETA = 'Vegeta'
    # Add More

## PLAYER CLASS

In [79]:
class Player:
    def __init__(self):
        self.initial_stages = 5
        self.life_deck = Deck()
        self.discard_pile = DiscardPile()
        self.main_personality = None
        self.hand = Hand()

    def draw_cards(self, num_cards):
        drawn_cards = self.life_deck.draw_cards(num_cards)
        self.hand.add_cards(drawn_cards)
        return drawn_cards
    
    def discard_cards(self, num_cards):
        discarded_cards = self.hand.get_cards()[:num_cards]
        self.hand.remove_cards(discarded_cards)
        self.discard_pile.add_cards_to_deck(discarded_cards)

    def play_card_from_hand(self, card):
        if card in self.hand.get_cards():
            self.hand.remove_card(card)
            card.play(self)

            
    def peek_cards(self, num_cards):
        return self.life_deck.peek_cards(num_cards)

    def shuffle_deck(self):
        self.life_deck.shuffle_deck()

    def look_through_deck(self, card_name):
        return self.life_deck.look_through_deck(card_name)

    def add_card_to_deck(self, card):
        self.life_deck.add_card_to_deck(card)

    def add_cards_to_deck(self, cards):
        self.life_deck.add_cards_to_deck(cards)

    def get_deck_size(self):
        return self.life_deck.get_deck_size()

    def get_discard_pile_size(self):
        return self.discard_pile.get_deck_size()

        
    def take_damage(self, damage):
        # Reduce life cards from the life deck based on the damage taken
        while damage > 0 and self.life_deck:
            self.life_deck.pop(0)
            damage -= 1

    def play_main_personality(self, card):
        # Check if the player already has a main personality card in play
        if self.main_personality is not None:
            print("A main personality card is already in play.")
            return False
        # Set the main personality card in play
        self.main_personality = card
        return True
    
    def play_card(self, card):
        if card.is_valid_to_play(self):
            card.play(self)
            print(f"{card.name} played successfully!")
        else:
            print(f"Not enough power stages to play {card.name}. Taking damage in life cards.")
            self.take_damage(card.life_card_damage)

## CARD CLASS

In [90]:
class Card:
    def __init__(self, 
                 name = '', # Card Name
                 text = '', # Card Text
                 card_type = None, # Card Type: Combat, Non-Combat, Dragon Ball, Personality, Mastery.
                 card_subtype = None, # Card Subtype: Energy Combat, Physical Combat, Setup, Drill, etc.
                 card_style = None,  # Saiyan Style, Namekian Style, etc.
                 named_character = '', # Named Character: Goku, Vegeta, etc.
                 effect_type = None, # Attack, Defense, Power, Constant Power
                 attack_type = None, # Physical Attack or Energy Attack
                 cost_power_stage = 0, # Power stages that must be paid to play card
                 cost_life_card = 0, # Life cards that must be discarded to play card
                 requirements = None, # Any other requirements that must be met to play card
                 in_play = False, 
                 damage_power_stage = 0, # Damage done in power stages
                 damage_life_card = 0, # Damage done in life cards
                 constant_effect = None, # Constant Combat Power
                 activated_ability = None, # Activated Ability, usually triggered once and discarded
                 card_number = None, # Card Number for collection purposes
                 card_rarity = None, # Card Rarity for collection purposes
                 set_name = None):  # Set Name for collection purposes
        self.name = name
        self.card_type = card_type
        self.card_subtype = card_subtype
        self.card_style = card_style
        self.named_character = named_character
        self.effect_type = effect_type
        self.attack_type = attack_type
        self.cost_power_stage = cost_power_stage 
        self.cost_life_card = cost_life_card
        self.requirements = requirements if requirements else {}
        self.in_play = in_play
        self.damage_power_stage = damage_power_stage
        self.damage_life_card = damage_life_card
        self.constant_effect = constant_effect
        self.activated_ability = activated_ability
        self.card_number = card_number
        self.card_rarity = card_rarity
        self.set_name = set_name
        
    def __str__(self):
        return self.name

    def is_valid_to_play(self, player):
        if self.card_type == CardType.COMBAT:
            return player.main_personality.current_power_stage_index >= self.cost_power_stage or player.life_deck.get_deck_size() >= self.cost_life_card
        else:
            return True

    def play(self, player):
        if self.card_type == CardType.COMBAT:
            if player.main_personality.current_power_stage_index >= self.cost_power_stage:
                player.main_personality.reduce_power_stages(self.cost_power_stage)
                self.in_play = True
            else:
                print("Insufficient power stages to play the card.")
        else:
            self.in_play = True
            # Handle non-combat card effects (setup, drills, dragon balls, allies, etc.)
            if self.constant_effect:
                # Apply constant effects of the card
                self.constant_effect.apply(player)
            # Add your code here to handle activated abilities of Non-Combat cards.
            
            
            
class ConstantEffect:
    def __init__(self, effect_text):
        self.effect_text = effect_text

    def apply(self, player):
        # Add your code here to apply the constant effect of the card to the player.
        pass


class ActivatedAbility:
    def __init__(self, ability_text, effect):
        self.ability_text = ability_text
        self.effect = effect

    def use(self, player):
        # Add your code here to handle the activated ability of the card.
        pass


# Other card attributes can be set separately based on the card type and its specific characteristics.
# For example:
# personality_card = Personality("Goku", power_stages=11)
# mastery_card = Card("Mastery", "Kamehameha Mastery")
# constant_effect_card = Card("Drill", "Training Drill", constant_effect=ConstantEffect("All attacks deal +1 damage."))
# activated_ability_card = Card("Non-Combat", "Super Combo", activated_ability=ActivatedAbility("Activate: Gain 3 power stages.", effect=lambda player: player.gain_power_stages(3)))

# You can add more attributes and methods to the `Card` class and related classes as needed, based on your game's mechanics.


### PERSONALITY CARD CLASS

In [91]:
class PersonalityCard(Card):
    def __init__(self, name, named_character, card_subtype, personality_level, power_up_rating, power_stages, card_type = CardType.PERSONALITY):
        super().__init__(name = name, 
                         named_character = named_character, 
                         card_type = card_type,
                         card_subtype = card_subtype) 
        self.power_stage_indices = range(0, len(power_stages))
        self.power_stage_values = power_stages
        self.current_power_stage_index = 5
        self.current_power_stage_value = self.power_stage_values[self.current_power_stage_index]
        self.personality_level = personality_level
        self.power_up_rating = power_up_rating
       
    # ... (other methods)

    def set_current_power_stage_index(self, current_stage_index):
        if current_stage_index < len(self.power_stage_indices):
            self.current_power_stage_index = current_stage_index
        else:
            raise ValueError("Invalid power stage index for this personality.")

    def get_current_power_stage_value(self):
        return self.power_stage_values[self.current_power_stage_index]

    def add_power_stages(self, stages):
        self.current_power_stage_index = min(self.current_power_stage_index + stages, len(self.power_stage_indices) - 1)
        self.current_power_stage_value = self.get_current_power_stage_value()

    def reduce_power_stages(self, stages):
        self.current_power_stage_index = max(self.current_power_stage_index - stages, 0)
        self.current_power_stage_value = self.get_current_power_stage_value()

In [92]:
# Create a sample personality card

In [93]:
SS_10_Goku = PersonalityCard(name = "Goku, the Hero", named_character = NamedCharacter.GOKU, card_subtype = PersonalityCardType.HERO, personality_level = 1, power_up_rating = 1, power_stages = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])

In [94]:
SS_10_Goku.name, SS_10_Goku.personality_level, SS_10_Goku.power_up_rating, SS_10_Goku.current_power_stage_index, SS_10_Goku.current_power_stage_value, SS_10_Goku.card_type.value, SS_10_Goku.card_subtype.value

('Goku, the Hero', 1, 1, 5, 500, 'Personality', 'Hero')

In [95]:
# create a sample energy attack card

In [112]:
# Create an energy combat card:
SS_01 = Card(name = 'Saiyan Energy Beam', card_type = CardType.COMBAT, card_subtype = CombatCardType.ENERGY_COMBAT, effect_type = EffectType.ATTACK, attack_type = AttackType.ENERGY_ATTACK, card_style = CardStyle.SAIYAN_STYLE, cost_power_stage = 2, damage_life_card = 4)
SS_02 = Card(name = 'Saiyan Energy Block', card_type = CardType.COMBAT, card_subtype = CombatCardType.ENERGY_COMBAT, effect_type = EffectType.DEFENSE, attack_type = AttackType.ENERGY_ATTACK, card_style = CardStyle.SAIYAN_STYLE)
SS_03 = Card(name = 'Saiyan Physical Attack', card_type = CardType.COMBAT, card_subtype = CombatCardType.PHYSICAL_COMBAT, effect_type = EffectType.ATTACK, attack_type = AttackType.PHYSICAL_ATTACK, card_style = CardStyle.SAIYAN_STYLE, cost_power_stage = 2, damage_power_stage= 4)
SS_04 = Card(name = 'Saiyan Physical Block', card_type = CardType.COMBAT, card_subtype = CombatCardType.PHYSICAL_COMBAT, effect_type = EffectType.DEFENSE, attack_type = AttackType.PHYSICAL_ATTACK, card_style = CardStyle.SAIYAN_STYLE)


In [97]:
SS_01.name, SS_01.card_type.value, SS_01.card_subtype.value, SS_01.card_style.value, SS_01.effect_type.value, SS_01.attack_type.value, SS_01.cost_power_stage, SS_01.cost_life_card, SS_01.damage_life_card, SS_01.damage_power_stage

('Saiyan Energy Beam',
 'Combat Card',
 'Energy Combat',
 'Saiyan Style',
 'Attack Type Card',
 'Energy Attack',
 2,
 0,
 4,
 0)

In [101]:
# Assuming you have a Player class with the main personality card in play
# Let's say Goku is the player
player1 = Player()

# Print Goku's current power stages before playing the card (it will be 0 as there's no main personality yet)
#print("Before playing the card - Current Power Stages:", player1.main_personality.get_current_power_stage_value())  # Output: 0

# Play the main personality card (Goku_ss_01)
player1.play_main_personality(Goku_ss_01)

# Print Goku's current power stages after playing the card
print("After playing the card - Current Power Stages:", player1.main_personality.get_current_power_stage_value())  # Output: e.g., 500 (assuming 500 is Goku_ss_01's power stage value)

# Add power stages (e.g., 2 stages)
player1.main_personality.add_power_stages(2)

# Print Goku's current power stages after adding stages
print("After adding stages - Current Power Stages:", player1.main_personality.get_current_power_stage_value())  # Output: e.g., 700 (assuming 700 is Goku_ss_01's power stage value after adding 2 stages)

# Reduce power stages (e.g., 3 stages)
player1.main_personality.reduce_power_stages(3)

# Print Goku's current power stages after reducing stages
print("After reducing stages - Current Power Stages:", player1.main_personality.get_current_power_stage_value())  # Output: e.g., 400 (assuming 400 is Goku_ss_01's power stage value after reducing 3 stages)


After playing the card - Current Power Stages: 0
After adding stages - Current Power Stages: 200
After reducing stages - Current Power Stages: 0


In [108]:
player1.main_personality.set_current_power_stage_index(9)
player1.main_personality.get_current_power_stage_value()

900

In [109]:
# Try to play the card (SS_01)
player1.play_card(SS_01)

# Print Goku's current power stages after playing SS_01 (if successful)
if player1.main_personality:
    print("After playing SS_01 - Current Power Stages:", player1.main_personality.get_current_power_stage_value())

# Print Goku's life deck after playing SS_01 (if unsuccessful)
print("Goku's Life Deck:", player1.life_deck)


Saiyan Energy Beam played successfully!
After playing SS_01 - Current Power Stages: 700
Goku's Life Deck: <__main__.Deck object at 0x000001A5F9BBAF10>


## GAME STEPS
gamesteps.py

In [77]:
from enum import Enum

class Phase(Enum):
    BEGINNING = 'Beginning Phase'
    PLANNING = 'Planning Phase'
    COMBAT = 'Combat Phase'
    END = 'End Phase'
    
    @property
    def steps(self):
        return{
            Phase.BEGINNING: (Step.TURN_BEGINS,
                              Step.RESET_COUNTERS,
                              Step.DRAW),
            Phase.PLANNING: (Step.PLAY_NONCOMBAT,
                             Step.POWER_UP,
                             Step.DECLARE_COMBAT),
            Phase.COMBAT: (Step.BEGINNING_OF_COMBAT,
                           Step.DEFENDER_DRAWS,
                           Step.ACTIVE_ATTACKS,
                           Step.DEFENDER_BLOCKS,
                           Step.DEFENDER_ATTACKS,
                           Step.ACTIVE_BLOCKS,
                           Step.END_OF_COMBAT),
            Phase.END: (Step.DISCARD,
                        Step.REJUVENATE,
                        Step.TURN_ENDS)
        }[self]
    
class Step(Enum):
    TURN_BEGINS = 'Beginning of Active Player Turn' # To trigger 'Beginning of Turn'
    RESET_COUNTERS = 'Reset Ability Counters' 
    DRAW = 'Draw Step for Active Player' # Draw 3 cards
    PLAY_NONCOMBAT = 'Non-Combat Card Step' # Play any Non-Combat Cards
    POWER_UP = 'Power Up Step' # Power up for Main Personality and Allies
    DECLARE_COMBAT = 'Declare Combat Step' # Declare or Pass Combat
    BEGINNING_OF_COMBAT = 'Beginning of Combat' # To trigger 'When Entering Combat'
    DEFENDER_DRAWS = 'Draw Step for Defending Player' # Defender draws 3 cards
    ACTIVE_ATTACKS = 'Active Player Attack Phase' # Active Player takes an attack action
    DEFENDER_BLOCKS = 'Defending Player Block Phase' # Defending Player takes block action
    DEFENDER_ATTACKS = 'Defending Player Attack Phase' # Defending Player takes attack action
    ACTIVE_BLOCKS = 'Active Player Block Phase' # Defending Player takes block action
    END_OF_COMBAT = 'End of Combat' # To trigger 'End of Combat'
    DISCARD = 'Both players discard down to '
    REJUVENATE = 'Rejuvenate' # Active Player rejuvenates if combat was not declared
    TURN_ENDS = 'End of Active Player Turn' # To trigger 'End of Turn'
    
    @property
    def phase(self):
        return{
            Step.TURN_BEGINS: Phase.BEGINNING,
            Step.RESET_COUNTERS: Phase.BEGINNING,
            Step.DRAW: Phase.BEGINNING,
            Step.PLAY_NONCOMBAT: Phase.PLANNING,
            Step.POWER_UP: Phase.PLANNING,
            Step.DECLARE_COMBAT: Phase.PLANNING,
            Step.BEGINNING_OF_COMBAT: Phase.COMBAT,
            Step.DEFENDER_DRAWS: Phase.COMBAT,
            Step.ACTIVE_ATTACKS: Phase.COMBAT,
            Step.DEFENDER_BLOCKS: Phase.COMBAT,
            Step.DEFENDER_ATTACKS: Phase.COMBAT,
            Step.ACTIVE_BLOCKS: Phase.COMBAT,
            Step.END_OF_COMBAT: Phase.COMBAT,
            Step.DISCARD: Phase.END,
            Step.REJUVENATE: Phase.END,
            Step.TURN_ENDS: Phase.END 
        }[self]

## ZONE CLASS
zone.py

In [110]:
class ZoneType(Enum):
    LIFE_DECK = 'Life Deck' # Main Deck where cards are drawn from
    DISCARD_PILE = 'Discard Pile' # Stores discarded cards through effect or damage
    BANISH_PILE = 'Banish Pile' # Stores removed from game cards through effects or damage
    HAND = 'Hand' # Cards stored in hand
    IN_PLAY = 'In Play' # Cards that are in play
    
class Zone():
    is_lifedeck = False
    is_inplay = False
    is_public = False
    
    def __init__(self, controller = None, elements: list = None):
        if elements is None:
            self.elements = []
        else:
            self.elements = elements
            for ele in elements:
                ele.controller = controller
            self.controller = controller
            if controller is not None:
                self.game = self.controller.game
                
    def __repr__(self):
        return 'zone.Zone %r controlled by %r len=%s\n%r' % (self.__class__.__name__,
                                                             self.controller, len(self), self.elements)

    def __str__(self):
        return '%s\'s %s (%s cards)\n%s' % (self.controller, 
                                            self.__class__.__name__,
                                            len(self), 
                                            [ele.name for ele in self.elements])

    def __len__(self):
        return len(self.elements)

    def __bool__(self):
        return bool(self.elements)

    def __getitem__(self, pos):
        return self.elements[pos]

    @property
    def isEmpty(self):
        return len(self) == 0
    
    def add(self, obj):
        if type(obj) is list:
            o.zone = self
            o.controller = self.controller
        self.elements.extend(obj)
        return obj
    
        obj.zone = self
        self.elements.append(obj)
        return obj

    def remove(self, obj):
        if type(obj) is list:
            return all([self.remove(o) for o in obj])
        
        try:
            self.elements.remove(obj)
            obj.zone = None
            return True
        except ValueError:
            return False
        
    def filter(self, characteristics = None, filter_func = None):
        found = set()
        if filter_func:
            for ele in self:
                if filter_func(ele):
                    found.add(ele)
        else:
            for ele in self:
                if ele.characteristics.satisfy(characteristics):
                    found.add(ele)
                    
        return found
    
    def count(self, characteristics = None, filter_func = None):
        return len(self.filter(characteristics, filter_func))
    
    def get_card_by_name(self, name):
        cards = self.filter(Card(name = name))
        if cards:
            return list(cards)[0]
        else:
            return None
        
    def pop(self, pos = -1):
        return self.elements.pop(pos)
    
    def clear(self):
        self.elements = []

class InPlay(Zone):
    zone_type = 'IN_PLAY'
    is_inplay = True
    is_public = True
    
    def add(self, obj, status_mode = )


## DECK CLASS

In [88]:
import random

class Deck:
    def __init__(self, cards=[]):
        self.cards = cards
        self.discard_pile = []

    def draw_cards(self, num_cards):
        drawn_cards = self.cards[:num_cards]
        self.cards = self.cards[num_cards:]
        return drawn_cards

    def discard_cards(self, num_cards):
        discarded_cards = self.cards[:num_cards]
        self.cards = self.cards[num_cards:]
        self.discard_pile.extend(discarded_cards)

    def peek_cards(self, num_cards):
        return self.cards[:num_cards]

    def shuffle_deck(self):
        random.shuffle(self.cards)

    def look_through_deck(self, card_name):
        found_cards = [card for card in self.cards if card.name == card_name]
        return found_cards

    def add_card_to_deck(self, card):
        self.cards.append(card)

    def add_cards_to_deck(self, cards):
        self.cards.extend(cards)

    def get_deck_size(self):
        return len(self.cards)

    def get_discard_pile_size(self):
        return len(self.discard_pile)


## HAND CLASS

In [74]:
class Hand:
    def __init__(self):
        self.cards = []

    def add_card(self, card):
        self.cards.append(card)

    def remove_card(self, card):
        self.cards.remove(card)

    def get_cards(self):
        return self.cards

    def is_empty(self):
        return len(self.cards) == 0

    def clear(self):
        self.cards = []


## DISCARD PILE CLASS

In [75]:
class DiscardPile:
    def __init__(self):
        self.cards = []

    def add_cards(self, cards):
        self.cards.extend(cards)

    def is_empty(self):
        return len(self.cards) == 0

    def clear(self):
        self.cards = []

In [ ]:
### card traits:
# can_be_discarded
# can_be_removed
# can_win_mppv
# can_raise_anger?
# can_lower_anger?
# can_use_endurance?

### Triggers:
# HIT: If attack successful
# Draw: when drawing
# Taking damage
# Played
# Discarded/banished from play/hand
# ENDURANCE (reduces life cards of damage)
# Critical Damage
# Personality Level Advanced / Lowered
# Anger raised or lowered
# Stopped an attack
# Rejuvenate
# Entering combat as attacker / defender
# Did not declare combat

 
### Card/Deck/Zones
# Draw (target draws from top/bottom of deck, discard pile, banish pile)
# Shuffle
# Search for specific card(s) in zones
# Rejuvenate (goes to bottom of life deck)
# Place on top of life deck
# Discard (goes to Discard Pile)
# Banish (goes to Banish Pile)
# Remain (stays in play to be used more times)
# Shuffle into life deck after use (can be combo of rejuv + shuffle to avoid more functions)
# Move to In Play from hand
# Move from In Play to hand
# Capture / Steal (move from opponent's in play to yours)

### Effects Functions
# Raise / Lower Anger
# Raise / Lower Personality level
# Raise / Lower Power Stages (either to min 0 or surplus lifecards)
# Prevent Damage (this turn, next turn or flaoting)
# Limit per deck
# Copy Attack
# Modify crit damage requirement
# Prevent use of card  types (cannot perform energy attacks, use events, etc)
# Modify card cost
# Modify damage until eoc (end of combat)
# End combat

### Conditions
# Dragon Balls in Play
# Ally in Play
# Non-Combat (setup or drill) Card in play
# Personality level
# Anger level


### Damage calc:
# Attack Table
# conditional damage (discard/banish/additional cost such as lower anger/personality)
# floating damage (from drills/cards/etc)

### Costs
# power stages
# life cards
# discard/banish card (from hand or in play) (specific or otherwise)
# lower/raise anger

###  Combat tracker (for conditions):
# crit damage?
# anger raised/lowered?
# personality advanced?
# physical/energy attack successful
# physical/energy/event combat cards played


In [ ]:
### TO-DO
# Make some cards
# Add these cards to a deck object
# Create a player object with the deck from above.
# See if we can draw, discard, shuffle, search, etc

# Fix the different sub zones:
